In [3]:
import os
import joblib
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import (
    f1_score, accuracy_score, precision_score, recall_score,
    classification_report, confusion_matrix, make_scorer
)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

OUTPUT_DIR = r"F:\Ai&ml\outputs"
MODEL_DIR = os.path.join(OUTPUT_DIR, "models")
RESULTS_DIR = os.path.join(OUTPUT_DIR, "results")
VIZ_DIR = os.path.join(OUTPUT_DIR, "visualization")

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(VIZ_DIR, exist_ok=True)

print("="*70)
print("LIVER CIRRHOSIS MODEL TRAINING PIPELINE")
print("="*70)
print(f"\nOutput directories:")
print(f"  • Models: {MODEL_DIR}")
print(f"  • Results: {RESULTS_DIR}")
print(f"  • Visualizations: {VIZ_DIR}")

def weighted_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

def macro_f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def evaluate_model(model, X_test, y_test, model_name="Model"):
    y_pred = model.predict(X_test)

    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'weighted_f1': f1_score(y_test, y_pred, average='weighted'),
        'macro_f1': f1_score(y_test, y_pred, average='macro'),
        'weighted_precision': precision_score(y_test, y_pred, average='weighted'),
        'weighted_recall': recall_score(y_test, y_pred, average='weighted'),
    }

    report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)

    return metrics, report, cm, y_pred

def plot_confusion_matrix(cm, model_name, save_path):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
                square=True, linewidths=1, linecolor='black')
    plt.title(f'Confusion Matrix - {model_name}', fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_model_comparison(results_dict, metric='weighted_f1'):
    models = list(results_dict.keys())
    scores = [results_dict[m]['cv_mean'] for m in models]
    std = [results_dict[m]['cv_std'] for m in models]

    plt.figure(figsize=(12, 6))
    bars = plt.bar(models, scores, yerr=std, capsize=5,
                   color='skyblue', edgecolor='black', linewidth=1.5)

    for bar, score in zip(bars, scores):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{score:.4f}', ha='center', va='bottom', fontweight='bold')

    plt.title(f'Model Comparison - Cross-Validation {metric.replace("_", " ").title()}',
              fontsize=16, fontweight='bold')
    plt.ylabel('Score', fontsize=12)
    plt.xlabel('Model', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1.1)
    plt.grid(axis='y', alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_DIR, f'model_comparison_{metric}.png'),
                dpi=300, bbox_inches='tight')
    plt.close()

def plot_feature_importance(model, feature_names, model_name, top_n=20):
    if hasattr(model, 'named_steps'):
        estimator = model.named_steps['model']
    else:
        estimator = model

    if hasattr(estimator, 'feature_importances_'):
        importances = estimator.feature_importances_
        indices = np.argsort(importances)[::-1][:top_n]

        plt.figure(figsize=(12, 8))
        plt.barh(range(top_n), importances[indices], color='teal')
        plt.yticks(range(top_n), [feature_names[i] for i in indices])
        plt.xlabel('Importance', fontsize=12)
        plt.title(f'Top {top_n} Feature Importances - {model_name}',
                 fontsize=16, fontweight='bold')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.savefig(os.path.join(VIZ_DIR, f'feature_importance_{model_name.replace(" ", "_")}.png'),
                   dpi=300, bbox_inches='tight')
        plt.close()

        importance_df = pd.DataFrame({
            'feature': [feature_names[i] for i in indices],
            'importance': importances[indices]
        })
        importance_df.to_csv(
            os.path.join(RESULTS_DIR, f'feature_importance_{model_name.replace(" ", "_")}.csv'),
            index=False
        )

def train_initial_models(X_train, y_train, X_test, y_test, feature_names):
    print("\n" + "="*70)
    print("STEP 1: TRAINING INITIAL MODELS")
    print("="*70)

    models = {
        'Random Forest': RandomForestClassifier(
            n_estimators=100, random_state=42, n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingClassifier(
            n_estimators=100, random_state=42
        ),
        'XGBoost': XGBClassifier(
            n_estimators=100, random_state=42,
            eval_metric='mlogloss', n_jobs=-1
        ),
        'LightGBM': LGBMClassifier(
            n_estimators=100, random_state=42,
            n_jobs=-1, verbose=-1
        ),
        'CatBoost': CatBoostClassifier(
            iterations=100, random_state=42,
            verbose=0, thread_count=-1
        )
    }

    unique, counts = np.unique(y_train, return_counts=True)
    print(f"\nTraining set class distribution:")
    for cls, cnt in zip(unique, counts):
        print(f"  Class {cls}: {cnt} samples ({cnt/len(y_train)*100:.1f}%)")

    use_smote = len(unique) > 1 and min(counts) < 50
    if use_smote:
        print("\n⚠️  Detected class imbalance. Applying SMOTE...")
        smote = SMOTE(random_state=42)
    else:
        print("\n✓ Balanced dataset. Skipping SMOTE...")
        smote = None

    results = {}
    trained_models = {}
    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for name, model in models.items():
        print(f"\n[{list(models.keys()).index(name) + 1}/{len(models)}] Training {name}...")

        if use_smote:
            pipeline = ImbPipeline(steps=[
                ('smote', smote),
                ('model', model)
            ])
        else:
            pipeline = ImbPipeline(steps=[('model', model)])

        cv_scores = cross_val_score(
            pipeline, X_train, y_train,
            cv=cv_strategy,
            scoring=make_scorer(weighted_f1),
            n_jobs=-1
        )

        pipeline.fit(X_train, y_train)

        metrics, report, cm, y_pred = evaluate_model(
            pipeline, X_test, y_test, name
        )

        results[name] = {
            'cv_mean': np.mean(cv_scores),
            'cv_std': np.std(cv_scores),
            'cv_scores': cv_scores.tolist(),
            'test_metrics': metrics,
            'classification_report': report
        }

        trained_models[name] = pipeline

        print(f"  ✓ CV F1 Score: {np.mean(cv_scores):.4f} (±{np.std(cv_scores):.4f})")
        print(f"  ✓ Test Accuracy: {metrics['accuracy']:.4f}")
        print(f"  ✓ Test F1 Score: {metrics['weighted_f1']:.4f}")

        plot_confusion_matrix(
            cm, name,
            os.path.join(VIZ_DIR, f'confusion_matrix_{name.replace(" ", "_")}.png')
        )

        plot_feature_importance(pipeline, feature_names, name)

    best_model_name = max(results, key=lambda x: results[x]['cv_mean'])
    best_model = trained_models[best_model_name]
    best_score = results[best_model_name]['cv_mean']

    print("\n" + "="*70)
    print(f"🏆 BEST MODEL: {best_model_name}")
    print(f"   CV F1 Score: {best_score:.4f}")
    print("="*70)

    joblib.dump(best_model, os.path.join(MODEL_DIR, 'best_initial_model.joblib'))
    joblib.dump(best_model_name, os.path.join(MODEL_DIR, 'best_model_name.joblib'))

    for name, model in trained_models.items():
        safe_name = name.replace(' ', '_').lower()
        joblib.dump(model, os.path.join(MODEL_DIR, f'{safe_name}_model.joblib'))

    with open(os.path.join(RESULTS_DIR, 'initial_model_results.json'), 'w') as f:
        json.dump(results, f, indent=4)

    summary_df = pd.DataFrame({
        'Model': list(results.keys()),
        'CV_Mean': [results[m]['cv_mean'] for m in results],
        'CV_Std': [results[m]['cv_std'] for m in results],
        'Test_Accuracy': [results[m]['test_metrics']['accuracy'] for m in results],
        'Test_F1': [results[m]['test_metrics']['weighted_f1'] for m in results]
    }).sort_values('CV_Mean', ascending=False)

    summary_df.to_csv(os.path.join(RESULTS_DIR, 'model_comparison_summary.csv'), index=False)

    plot_model_comparison(results)

    return best_model, best_model_name, results, trained_models

def hyperparameter_tuning(X_train, y_train, X_test, y_test,
                         feature_names, base_model_name=None):
    print("\n" + "="*70)
    print("STEP 2: HYPERPARAMETER TUNING")
    print("="*70)

    param_grids = {
        'XGBoost': {
            'model__n_estimators': [100, 200, 300, 500],
            'model__max_depth': [3, 5, 7, 9],
            'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
            'model__subsample': [0.7, 0.8, 0.9, 1.0],
            'model__colsample_bytree': [0.7, 0.8, 0.9, 1.0],
            'model__gamma': [0, 0.1, 0.2, 0.3],
            'model__min_child_weight': [1, 3, 5]
        },
        'LightGBM': {
            'model__n_estimators': [100, 200, 300, 500],
            'model__max_depth': [3, 5, 7, 9, -1],
            'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
            'model__num_leaves': [15, 31, 63, 127],
            'model__subsample': [0.7, 0.8, 0.9, 1.0],
            'model__colsample_bytree': [0.7, 0.8, 0.9, 1.0]
        },
        'Random Forest': {
            'model__n_estimators': [100, 200, 300, 500],
            'model__max_depth': [10, 20, 30, None],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': ['sqrt', 'log2', None]
        },
        'Gradient Boosting': {
            'model__n_estimators': [100, 200, 300],
            'model__max_depth': [3, 5, 7],
            'model__learning_rate': [0.01, 0.05, 0.1],
            'model__subsample': [0.7, 0.8, 0.9, 1.0],
            'model__min_samples_split': [2, 5, 10]
        },
        'CatBoost': {
            'model__iterations': [100, 200, 300, 500],
            'model__depth': [4, 6, 8, 10],
            'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
            'model__l2_leaf_reg': [1, 3, 5, 7, 9]
        }
    }

    if base_model_name and base_model_name in param_grids:
        model_name = base_model_name
    else:
        model_name = 'XGBoost'

    print(f"\n🔍 Tuning {model_name}...")

    if model_name == 'XGBoost':
        base_model = XGBClassifier(random_state=42, eval_metric='mlogloss', n_jobs=-1)
    elif model_name == 'LightGBM':
        base_model = LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1)
    elif model_name == 'Random Forest':
        base_model = RandomForestClassifier(random_state=42, n_jobs=-1)
    elif model_name == 'Gradient Boosting':
        base_model = GradientBoostingClassifier(random_state=42)
    elif model_name == 'CatBoost':
        base_model = CatBoostClassifier(random_state=42, verbose=0, thread_count=-1)

    unique, counts = np.unique(y_train, return_counts=True)
    use_smote = len(unique) > 1 and min(counts) < 50

    if use_smote:
        smote = SMOTE(random_state=42)
        pipeline = ImbPipeline(steps=[('smote', smote), ('model', base_model)])
    else:
        pipeline = ImbPipeline(steps=[('model', base_model)])

    param_grid = param_grids[model_name]

    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_grid,
        n_iter=30,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring=make_scorer(weighted_f1),
        random_state=42,
        n_jobs=-1,
        verbose=1
    )

    print(f"\n⏳ Running RandomizedSearchCV with {random_search.n_iter} iterations...")
    random_search.fit(X_train, y_train)

    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    best_cv_score = random_search.best_score_

    print(f"\n✅ Best CV F1 Score: {best_cv_score:.4f}")
    print(f"\n📋 Best Parameters:")
    for param, value in best_params.items():
        print(f"  • {param}: {value}")

    metrics, report, cm, y_pred = evaluate_model(
        best_model, X_test, y_test, f"{model_name} (Tuned)"
    )

    print(f"\n📊 Test Set Performance:")
    print(f"  • Accuracy: {metrics['accuracy']:.4f}")
    print(f"  • Weighted F1: {metrics['weighted_f1']:.4f}")
    print(f"  • Macro F1: {metrics['macro_f1']:.4f}")

    joblib.dump(best_model, os.path.join(MODEL_DIR, 'best_tuned_model.joblib'))

    with open(os.path.join(RESULTS_DIR, 'best_hyperparameters.json'), 'w') as f:
        json.dump(best_params, f, indent=4)

    tuning_results = {
        'model_name': model_name,
        'best_cv_score': best_cv_score,
        'best_params': best_params,
        'test_metrics': metrics,
        'cv_results': {
            'mean_test_score': random_search.cv_results_['mean_test_score'].tolist(),
            'std_test_score': random_search.cv_results_['std_test_score'].tolist(),
            'params': [str(p) for p in random_search.cv_results_['params']]
        }
    }

    with open(os.path.join(RESULTS_DIR, 'tuning_results.json'), 'w') as f:
        json.dump(tuning_results, f, indent=4)

    plot_confusion_matrix(
        cm, f"{model_name} (Tuned)",
        os.path.join(VIZ_DIR, f'confusion_matrix_tuned_{model_name.replace(" ", "_")}.png')
    )

    plot_feature_importance(best_model, feature_names, f"{model_name} (Tuned)")

    cv_results_df = pd.DataFrame({
        'iteration': range(len(random_search.cv_results_['mean_test_score'])),
        'mean_score': random_search.cv_results_['mean_test_score'],
        'std_score': random_search.cv_results_['std_test_score']
    }).sort_values('mean_score', ascending=False)

    plt.figure(figsize=(12, 6))
    plt.errorbar(cv_results_df['iteration'], cv_results_df['mean_score'],
                yerr=cv_results_df['std_score'], fmt='o-', capsize=3, alpha=0.7)
    plt.axhline(y=best_cv_score, color='r', linestyle='--',
                label=f'Best Score: {best_cv_score:.4f}')
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('CV F1 Score', fontsize=12)
    plt.title(f'Hyperparameter Tuning Results - {model_name}',
              fontsize=16, fontweight='bold')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_DIR, 'hyperparameter_tuning_results.png'),
               dpi=300, bbox_inches='tight')
    plt.close()

    return best_model, best_params, tuning_results

if __name__ == "__main__":
    start_time = datetime.now()

    print("\n" + "="*70)
    print("LOADING DATA")
    print("="*70)

    DATA_DIR = r"F:\Ai&ml\outputs\datasets"
    X_train = joblib.load(os.path.join(DATA_DIR, 'X_train.joblib'))
    y_train = joblib.load(os.path.join(DATA_DIR, 'y_train.joblib'))
    X_test = joblib.load(os.path.join(DATA_DIR, 'X_test.joblib'))
    y_test = joblib.load(os.path.join(DATA_DIR, 'y_test.joblib'))
    feature_names = joblib.load(os.path.join(DATA_DIR, 'feature_names.joblib'))

    print(f"\n✓ Data loaded successfully!")
    print(f"  • Training samples: {len(X_train)}")
    print(f"  • Test samples: {len(X_test)}")
    print(f"  • Features: {len(feature_names)}")

    best_initial_model, best_model_name, initial_results, all_models = train_initial_models(
        X_train, y_train, X_test, y_test, feature_names
    )

    best_tuned_model, best_params, tuning_results = hyperparameter_tuning(
        X_train, y_train, X_test, y_test, feature_names, best_model_name
    )

    print("\n" + "="*70)
    print("FINAL MODEL COMPARISON")
    print("="*70)

    initial_score = initial_results[best_model_name]['test_metrics']['weighted_f1']
    tuned_score = tuning_results['test_metrics']['weighted_f1']
    improvement = ((tuned_score - initial_score) / initial_score) * 100

    print(f"\n{best_model_name}:")
    print(f"  • Initial Test F1: {initial_score:.4f}")
    print(f"  • Tuned Test F1: {tuned_score:.4f}")
    print(f"  • Improvement: {improvement:+.2f}%")

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    final_report = {
        'execution_time_seconds': duration,
        'best_initial_model': best_model_name,
        'initial_best_score': initial_score,
        'tuned_best_score': tuned_score,
        'improvement_percent': improvement,
        'timestamp': end_time.strftime('%Y-%m-%d %H:%M:%S')
    }

    with open(os.path.join(RESULTS_DIR, 'final_training_report.json'), 'w') as f:
        json.dump(final_report, f, indent=4)

    print("\n" + "="*70)
    print("✅ TRAINING PIPELINE COMPLETED!")
    print("="*70)
    print(f"\n⏱️  Total execution time: {duration:.2f} seconds")
    print(f"\n📁 All outputs saved to: {OUTPUT_DIR}")
    print(f"  • Models: {MODEL_DIR}")
    print(f"  • Results: {RESULTS_DIR}")
    print(f"  • Visualizations: {VIZ_DIR}")
    print("\n" + "="*70)
    print("By_OwenXAGK")

LIVER CIRRHOSIS MODEL TRAINING PIPELINE

Output directories:
  • Models: F:\Ai&ml\outputs\models
  • Results: F:\Ai&ml\outputs\results
  • Visualizations: F:\Ai&ml\outputs\visualization

LOADING DATA

✓ Data loaded successfully!
  • Training samples: 20000
  • Test samples: 5000
  • Features: 20

STEP 1: TRAINING INITIAL MODELS

Training set class distribution:
  Class 0: 6612 samples (33.1%)
  Class 1: 6753 samples (33.8%)
  Class 2: 6635 samples (33.2%)

✓ Balanced dataset. Skipping SMOTE...

[1/5] Training Random Forest...
  ✓ CV F1 Score: 0.9495 (±0.0027)
  ✓ Test Accuracy: 0.9520
  ✓ Test F1 Score: 0.9520

[2/5] Training Gradient Boosting...
  ✓ CV F1 Score: 0.8528 (±0.0047)
  ✓ Test Accuracy: 0.8426
  ✓ Test F1 Score: 0.8429

[3/5] Training XGBoost...
  ✓ CV F1 Score: 0.9594 (±0.0005)
  ✓ Test Accuracy: 0.9598
  ✓ Test F1 Score: 0.9598

[4/5] Training LightGBM...
  ✓ CV F1 Score: 0.9565 (±0.0013)
  ✓ Test Accuracy: 0.9558
  ✓ Test F1 Score: 0.9558

[5/5] Training CatBoost...
  ✓ 